In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)

In [3]:
# base_path = os.path.join('/orange/ufdatastudios/dj.brinkley/predictions/data/')
financial_full_path = os.path.join(notebook_dir, '../data/financial_phrase_bank/all_data-adjusted_header.csv')
df = DataProcessing.load_from_file(financial_full_path, 'csv')
df

,sentiment,sentence
0,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."
1,neutral,"Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said ."
2,negative,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported ."
3,positive,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
4,positive,"According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales ."
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower in London Monday as a rebound in bank stocks failed to offset broader weakness for the FTSE 100 .
4842,neutral,"Rinkuskiai 's beer sales fell by 6.5 per cent to 4.16 million litres , while Kauno Alus ' beer sales jumped by 6.9 per cent to 2.48 million litres ."
4843,negative,"Operating profit fell to EUR 35.4 mn from EUR 68.8 mn in 2007 , including vessel sales gain of EUR 12.3 mn ."
4844,negative,"Net sales of the Paper segment decreased to EUR 221.6 mn in the second quarter of 2009 from EUR 241.1 mn in the second quarter of 2008 , while operating profit excluding non-recurring items rose to EUR 8.0 mn from EUR 7.6 mn ."


In [4]:
df

,sentiment,sentence
0,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."
1,neutral,"Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said ."
2,negative,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported ."
3,positive,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
4,positive,"According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales ."
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower in London Monday as a rebound in bank stocks failed to offset broader weakness for the FTSE 100 .
4842,neutral,"Rinkuskiai 's beer sales fell by 6.5 per cent to 4.16 million litres , while Kauno Alus ' beer sales jumped by 6.9 per cent to 2.48 million litres ."
4843,negative,"Operating profit fell to EUR 35.4 mn from EUR 68.8 mn in 2007 , including vessel sales gain of EUR 12.3 mn ."
4844,negative,"Net sales of the Paper segment decreased to EUR 221.6 mn in the second quarter of 2009 from EUR 241.1 mn in the second quarter of 2008 , while operating profit excluding non-recurring items rose to EUR 8.0 mn from EUR 7.6 mn ."


In [5]:
sentences = DataProcessing.df_to_list(df, 'sentence')
sentences = sentences[:33]

In [6]:
tgmf = TextGenerationModelFactory()


# Groq Cloud (https://console.groq.com/docs/overview)
llama_318b_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant') 
llama_3370b_versatile_generation_model = tgmf.create_instance('llama-3.3-70b-versatile')  
gpt_model = tgmf.create_instance('gpt-oss-120b')  

models  = [llama_318b_instant_generation_model]
# models  = [llama_318b_instant_generation_model, gpt_model]
# models = [gpt_model]

In [7]:
prompt = """

Role:
You are a linguist expert. You are acting as a prediction detector. Your task is to identify if a given sentence is a prediction about the future.

Background:
A prediction is a statement about what someone thinks will happen in the future.
Examples of predictions:
- "It will rain tomorrow." (Yes)
- "The stock market is expected to rise next quarter." (Yes)
- "I am going to the store." (No)
-“Lakers will win the championship. ”(Yes)

A prediction may contain: source, target, date, outcome. See the above definition.
"""

In [8]:
output = {}
for sentence in tqdm(sentences):
        for model in models:
                updated_prompt = f"{prompt}\n Task: Analyze the following sentence {sentence} and determine if it is a prediction. Explain your reasoning before providing a 'Yes' or 'No' answer."
                input_prompt = model.user(updated_prompt)
                        # print(input_prompt)  

                raw_text_llm_generation = model.chat_completion([input_prompt])
                # print(raw_text_llm_generation)

                for line in raw_text_llm_generation.split("\n"):
                        output[sentence] = line

100%|██████████| 33/33 [01:53<00:00,  3.45s/it]


In [9]:
output

{'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .': 'The sentence is not a prediction because it does not express a statement about what someone thinks will happen in the future.',
 'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .': 'Answer: Yes',
 'The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .': 'Answer: No',
 'With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .': 'Therefore, the answer is: Yes',
 "According to the company 's updated 

In [10]:
for key, value in output.items():
    print("=============")
    print(f"Key: {key}")
    print(f"\tLabel:{value}")

Key: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
	Label:The sentence is not a prediction because it does not express a statement about what someone thinks will happen in the future.
Key: Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .
	Label:Answer: Yes
Key: The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .
	Label:Answer: No
Key: With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
	Label:Therefore, the answer is: Yes
Key: According to 

In [11]:
output_df = pd.DataFrame.from_dict(output, orient='index').reset_index()
output_df.columns = ['sentence', 'classification']
output_df

,sentence,classification
0,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",The sentence is not a prediction because it does not express a statement about what someone thinks will happen in the future.
1,"Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .",Answer: Yes
2,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",Answer: No
3,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,"Therefore, the answer is: Yes"
4,"According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .",Answer: Yes
5,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,Answer: No
6,"For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .",Answer: No
7,"In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .",**No**
8,Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .,Answer: No
9,"Operating profit totalled EUR 21.1 mn , up from EUR 18.6 mn in 2007 , representing 9.7 % of net sales .","Therefore, the answer is: No."
